In [1]:
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
import csv
import json
import os 
import string

In [3]:
import sys
sys.path.insert(0, '/home/bibek/projects/DEEPL')

import helpers
import classifier

In [4]:
df = pd.read_csv('nlp/nlp_out.csv', header = 0)

#convert json string columns to json
df[df.filter(like="_j").columns] = df.filter(like="_j").applymap(lambda x : json.loads(x))

#change column names
for v in df.filter(like="_j"):
    df = df.rename(columns = {v : '_'.join(v.split('_')[:-1])})
df

,onedim,twodim,reliability,severity,demo_groups,specific_needs,aff_groups,geo,info_date,excerpt,has_image,lead_text,lead_id,lead_url,event
0,{'Population Profile': ['Demographic Profile']},None,Usually,Situation of Concern,[Children (5 to 11 years old)],None,[Affected],None,2016-11-15,"894,057 children are among the 2.1 million aff...",True,None,4,http://reliefweb.int/sites/reliefweb.int/files...,Test Event
1,None,"[{'sector': 'Nutrition', 'subsectors': ['Breas...",Usually,Severe Conditions,[Infants/toddlers (< 5 years old)],None,[Affected],None,2016-11-15,"112,500 children under five are at risk of acu...",True,None,4,http://reliefweb.int/sites/reliefweb.int/files...,Test Event
2,None,"[{'sector': 'Health', 'subsectors': ['Health s...",Usually,Severe Conditions,None,None,[Affected],None,2016-11-15,"2,271 cholera cases suspected between 4-19 Oct...",True,None,4,http://reliefweb.int/sites/reliefweb.int/files...,Test Event
3,None,"[{'sector': 'Food', 'subsectors': None, 'pilla...",Usually,Severe Conditions,None,None,[Affected],None,2016-11-15,"806,000 people need urgent food assistance.",True,None,4,http://reliefweb.int/sites/reliefweb.int/files...,Test Event
4,None,"[{'sector': 'Health', 'subsectors': None, 'pil...",Usually,Situation of Concern,None,None,[Affected],None,2016-11-15,36 health facilities destroyed.,True,None,4,http://reliefweb.int/sites/reliefweb.int/files...,Test Event
5,{'Population Profile': ['Humanitarian Profile']},None,Usually,Situation of Major Concern,None,None,[Affected],None,2016-11-15,2.1 million Affected people,True,None,4,http://reliefweb.int/sites/reliefweb.int/files...,Test Event
6,None,"[{'sector': 'Cross', 'subsectors': None, 'pill...",Usually,Situation of Major Concern,None,None,[Affected],None,2016-11-15,1.4 million People need aid \r\n,True,None,4,http://reliefweb.int/sites/reliefweb.int/files...,Test Event
7,{'Context': ['Overview']},None,Usually,Situation of Major Concern,None,None,[Affected],None,2016-11-15,"On 4 October, Hurricane Matthew violently stru...",True,None,4,http://reliefweb.int/sites/reliefweb.int/files...,Test Event
8,{'Population Profile': ['Humanitarian Profile']},None,Usually,Situation of Major Concern,None,None,[Affected],None,2016-11-15,The latest figures from the governmental Direc...,True,None,4,http://reliefweb.int/sites/reliefweb.int/files...,Test Event
9,{'Humanitarian access': ['Physical Constraints']},None,Usually,Situation of Major Concern,None,None,[Affected],None,2016-11-15,Though access continues to be gained to more a...,True,None,4,http://reliefweb.int/sites/reliefweb.int/files...,Test Event


In [5]:
entries_excerpts = {}
for i, x in df.iterrows():
    if x['twodim']:
        for y in x['twodim']:
            key = y['sector'].lower()
            entries_excerpts[key] = entries_excerpts.get(key, []) + [x['excerpt']]

In [6]:
from helpers import functional, common
pre_processor = functional.compose(
    common.rm_punc_not_nums,
    str.lower,
    str,
)
processed_excerpts = {k:[pre_processor(x) for x in v] for k, v in entries_excerpts.items()}

In [15]:
## NOW USE TF-IDF on each of them
from classifier import tf_idf

tokenize = lambda doc: doc.lower().split(" ")
docs = processed_excerpts['wash'][:29]
print(docs)
rels = tf_idf.relevant_terms(list(map(tokenize, docs)))
print(rels)

['in addition to 775000 from the disbursed emergency funds transferred directly to communities the government has also shipped more than 46854mt of food including 165 tons of rice and 130000 of water ', 'an estimated 36900 people in grandanse and sud are targeted for health and wash services including 3410 children in institutions who require wash and nutrition assistance ', 'one of the damaged facilities in western rural aleppo was a frontline trauma hospital another was a secondline trauma hospital on a monthly basis all three hospitals were providing more than 10 000 consultations and more than 1500 major surgeries these attacks have seriously reduced the availability of health services in northern syria and resulted in a major disruption of referrals of critically injured and ill patients', ' the 2016 summer paralympic games closed in rio de janeiro brazil on 18 september who assesses the individual risk of zika virus infection in travellers returning from the paralympic games as l

In [14]:
## USE RAKE
import rake_nltk
rake = rake_nltk.Rake()
wash_text = ' '.join(processed_excerpts['wash'])
mytext = '''
This paper introduces a novel and domain-independent method for automatically extracting keywords, as sequences of one or more words, from individual documents. We describe the methods configuration parameters and algorithm, and present an evaluation on a benchmark corpus of technical abstracts. We also present a method for generating lists of stop words for specific corpora and domains, and evaluate its ability to improve keyword extraction on the benchmark corpus.
'''
#rake.extract_keywords_from_text(wash_text)
rake.extract_keywords_from_text(mytext)

list(filter(lambda x: len(x[1].split())<=2, rake.get_ranked_phrases_with_scores()))

[(4.0, 'technical abstracts'),
 (4.0, 'specific corpora'),
 (4.0, 'paper introduces'),
 (4.0, 'individual documents'),
 (4.0, 'generating lists'),
 (4.0, 'benchmark corpus'),
 (3.5, 'stop words'),
 (3.5, 'independent method'),
 (3.5, 'also present'),
 (1.5, 'words'),
 (1.5, 'present'),
 (1.5, 'method'),
 (1.0, 'sequences'),
 (1.0, 'one'),
 (1.0, 'novel'),
 (1.0, 'evaluation'),
 (1.0, 'evaluate'),
 (1.0, 'domains'),
 (1.0, 'domain'),
 (1.0, 'describe'),
 (1.0, 'algorithm'),
 (1.0, 'ability')]

In [ ]:
## COMPARE TF-IDF and RAKE